In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

DATA COLLECTION

Datos de Yahoo Finance

In [2]:
sp500 = yf.Ticker("^GSPC")
gold = yf.Ticker("^GC=F")
oil= yf.Ticker("CL=F")
brent= yf.Ticker("BZ=F")

Preparación y limpieza del Sp500

Para el Sp500 utilizo los datos a partir del año 2000, ya que en años anteriores la composición por industria del SP500 era bastante diferente. A partir de ese año hay importnate participación del sector teconología.

In [3]:
start_date = '2000-01-01'
end_date = '2024-04-26'

sp500 = sp500.history(period="max")
sp500 =sp500.loc[start_date:end_date].copy ()
oil = oil.history(period="max")
oil = oil.loc[start_date:end_date].copy ()
brent = brent.history(period="max") 
brent = brent.loc[start_date:end_date:].copy ()


In [4]:
sp500.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-03 00:00:00-05:00,1469.250000,1478.000000,1438.359985,1455.219971,931800000,0.0,0.0
2000-01-04 00:00:00-05:00,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,0.0,0.0
2000-01-05 00:00:00-05:00,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,0.0,0.0
2000-01-06 00:00:00-05:00,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,0.0,0.0
2000-01-07 00:00:00-05:00,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,0.0,0.0


In [5]:
sp500 = sp500.drop(columns=['Dividends', 'Stock Splits'])
sp500.index= sp500.index.tz_localize(None) # remove timezone

In [6]:
#Hago lo mismo con el resto de los activos
oil = oil.drop(columns=['Dividends', 'Stock Splits', 'Open', 'High', 'Low', 'Volume'])
oil.index= oil.index.tz_localize(None) # remove timezone
oil.head()

,Close
Date,
2000-08-23,32.049999
2000-08-24,31.629999
2000-08-25,32.049999
2000-08-28,32.869999
2000-08-29,32.720001


In [7]:
# Renombo la columna 'Close' a 'Close_oil' en el DataFrame 'oil' para que no se confunda con la del S&P 500
oil.rename(columns={'Close': 'Close_oil'}, inplace=True)

# Muestra las primeras filas para verificar el cambio
print(oil.head())


            Close_oil
Date                 
2000-08-23  32.049999
2000-08-24  31.629999
2000-08-25  32.049999
2000-08-28  32.869999
2000-08-29  32.720001


In [8]:
# Asumiendo que 'df' es tu DataFrame y 'oil' es la columna con los datos de precios de petróleo
oil['oil_daily_changes'] = oil['Close_oil'].pct_change() * 100  # Multiplicar por 100 para convertir a porcentaje


Calculo de las variables para análisis técnico

In [9]:
#Calculo la media 
# Periodos para las medias móviles
moving_avg_periods = [5,10, 20,200]

# Calcular y asignar cada media móvil utilizando un bucle
for period in moving_avg_periods:
    sp500[f'{period}d_moving_avg'] = sp500['Close'].rolling(window=period).mean()

# Mostrar las primeras filas para verificar las nuevas columnas
sp500.head()



,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg
Date,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN


In [10]:
#Calculo el promedio móvil ponderado de los ultimas 2 semanas

# Definimos los pesos decrecientes para los últimos 10 días hábiles.
weights = np.arange(1, 11)[::-1]  # Esto crea un array de numpy: [10, 9, ..., 2, 1]

# Calculamos el WMA para cada ventana de 10 días hábiles.
# Usamos apply para aplicar una función personalizada que calcula el WMA de manera vectorizada.
sp500['10d_wma'] = sp500['Close'].rolling(window=10).apply(lambda x: np.dot(x, weights) / weights.sum(), raw=True)

# Ahora, el DataFrame 'sp500' contiene una nueva columna '10d_wma' con el WMA de 10 días.
# Muestra los primeros registros para verificar.
print(sp500[['Close', '10d_wma']].head(20))


                  Close      10d_wma
Date                                
2000-01-03  1455.219971          NaN
2000-01-04  1399.420044          NaN
2000-01-05  1402.109985          NaN
2000-01-06  1403.449951          NaN
2000-01-07  1441.469971          NaN
2000-01-10  1457.599976          NaN
2000-01-11  1438.560059          NaN
2000-01-12  1432.250000          NaN
2000-01-13  1449.680054          NaN
2000-01-14  1465.150024  1428.005995
2000-01-18  1455.140015  1424.235638
2000-01-19  1455.900024  1431.637631
2000-01-20  1445.569946  1439.340725
2000-01-21  1441.359985  1447.489462
2000-01-24  1401.530029  1447.999288
2000-01-25  1410.030029  1444.711477
2000-01-26  1404.089966  1444.258740
2000-01-27  1398.560059  1444.340743
2000-01-28  1360.160034  1439.626008
2000-01-31  1394.459961  1430.813277


In [11]:
# Calcular el momentum como el cambio día a día en la columna 'Close'
sp500['Momentum'] = sp500['Close'].diff()

print(sp500.head())  # Mostrar las primeras filas para verificar la nueva columna

                   Open         High          Low        Close      Volume  \
Date                                                                         
2000-01-03  1469.250000  1478.000000  1438.359985  1455.219971   931800000   
2000-01-04  1455.219971  1455.219971  1397.430054  1399.420044  1009000000   
2000-01-05  1399.420044  1413.270020  1377.680054  1402.109985  1085500000   
2000-01-06  1402.109985  1411.900024  1392.099976  1403.449951  1092300000   
2000-01-07  1403.449951  1441.469971  1400.729980  1441.469971  1225200000   

            5d_moving_avg  10d_moving_avg  20d_moving_avg  200d_moving_avg  \
Date                                                                         
2000-01-03            NaN             NaN             NaN              NaN   
2000-01-04            NaN             NaN             NaN              NaN   
2000-01-05            NaN             NaN             NaN              NaN   
2000-01-06            NaN             NaN             NaN      

El Stochastic %K es una medida de la posición relativa del precio de cierre actual en relación con el rango alto-bajo durante un número específico de períodos pasados. La fórmula para calcular el Stochastic %K es la siguiente:

% K = [(C-LLn)/(HHn-LLn)]x 100� 

donde:


C es el precio de cierre más reciente,

LLn es el precio más bajo de los prcios bajos (Low) en los últimos n períodos,
Hn  es el precio más alto de lso pecios altos (High) en los últimos n períodos,
n es el número de períodos utilizados para el cálculo.
El valor de n varía según el análisis, pero los valores comunes incluyen 14, 20 o cualquier otro valor dependiendo de la estrategia de trading

In [12]:
# Calcular el Stochastic %K para cada fila basado en los 20 días laborables previos
# Crear una función para encapsular el cálculo
#def calculate_stochastic_k(row_index, df):
    #if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        #return None
    #else:
        #period = df['Close'][row_index-19:row_index+1]  # Seleccionar los últimos 20 registros hasta el índice actual
        #low_20 = period.min()
        #high_20 = period.max()
        #close_recent = df['Close'].iloc[row_index]
        #stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        #return stochastic_k

# Aplicar la función a cada fila
# Nota: Esto puede ser lento para DataFrames muy grandes debido al enfoque iterativo
#sp500['Stochastic_%K'] = [calculate_stochastic_k(i, sp500) for i in range(len(sp500))]

#print(sp500.tail())  # Mostrar las últimas filas para verificar la nueva columna
#sp500.head


# Crear una función para encapsular el cálculo del Stochastic %K
def calculate_stochastic_k(row_index, df):
    if row_index < 19:  # Asegurar que haya suficientes datos para los primeros 19 registros
        return None
    else:
        # Seleccionar los últimos 20 registros hasta el índice actual para las columnas 'High' y 'Low'
        high_period = df['High'][row_index-19:row_index+1]
        low_period = df['Low'][row_index-19:row_index+1]
        low_20 = low_period.min()
        high_20 = high_period.max()
        close_recent = df['Close'].iloc[row_index]
        
        # Calcular el Stochastic %K
        stochastic_k = ((close_recent - low_20) / (high_20 - low_20)) * 100
        return stochastic_k

# Aplicar la función a cada fila
sp500['Stochastic_%K'] = [calculate_stochastic_k(i, sp500) for i in range(len(sp500))]

# Mostrar las últimas filas para verificar la nueva columna
print(sp500.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-22  4987.330078  5038.839844  4969.399902  5010.600098  3820250000   
2024-04-23  5028.850098  5076.120117  5027.959961  5070.549805  3751400000   
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   

            5d_moving_avg  10d_moving_avg  20d_moving_avg  200d_moving_avg  \
Date                                                                         
2024-04-22    5012.514063     5081.741064     5147.851562      4677.076304   
2024-04-23    5016.341992     5067.805029     5140.469556      4680.434302   
2024-04-24    5026.225977     5058.904004     5133.872046      4683.744802   
2024-04-25    5033.685938     5043.839990     5123.868530      

El indicador Stochastic %D es una media móvil del indicador Stochastic %K. Usualmente se calcula como la media móvil simple de los últimos 3 valores de %K. Para calcular el %D para las últimas 20 observaciones de tu DataFrame sp500, primero necesitaríamos calcular el %K como se discutió anteriormente, y luego aplicar una media móvil sobre esos valores de %K.

In [13]:

# Calcular el Stochastic %D como la media móvil simple de 3 días de %K
sp500['Stochastic_%D'] = sp500['Stochastic_%K'].rolling(window=3).mean()

# Si solo necesitas las últimas 20 observaciones para el %D, puedes hacer lo siguiente:
last_20_stochastic_D = sp500['Stochastic_%D'][-20:]

print(last_20_stochastic_D)
sp500.head(9)

Date
2024-04-01    92.882218
2024-04-02    85.479659
2024-04-03    76.922842
2024-04-04    57.726515
2024-04-05    55.575326
2024-04-08    53.829101
2024-04-09    64.992990
2024-04-10    54.961573
2024-04-11    53.283501
2024-04-12    35.318808
2024-04-15    25.095684
2024-04-16     7.141325
2024-04-17     5.118661
2024-04-18     4.821237
2024-04-19     4.569616
2024-04-22     8.741723
2024-04-23    20.099123
2024-04-24    31.278398
2024-04-25    35.328193
2024-04-26    38.522920
Name: Stochastic_%D, dtype: float64


,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D
Date,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,NaN,-55.799927,NaN,NaN
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,NaN,2.689941,NaN,NaN
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,NaN,1.339966,NaN,NaN
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,NaN,38.020020,NaN,NaN
2000-01-10,1441.469971,1464.359985,1441.469971,1457.599976,1064800000,1420.809985,NaN,NaN,NaN,NaN,16.130005,NaN,NaN
2000-01-11,1457.599976,1458.660034,1434.420044,1438.560059,1014000000,1428.637988,NaN,NaN,NaN,NaN,-19.039917,NaN,NaN
2000-01-12,1438.560059,1442.599976,1427.079956,1432.250000,974600000,1434.665991,NaN,NaN,NaN,NaN,-6.310059,NaN,NaN
2000-01-13,1432.250000,1454.199951,1432.250000,1449.680054,1030400000,1443.912012,NaN,NaN,NaN,NaN,17.430054,NaN,NaN



El Índice de Fuerza Relativa (RSI, por sus siglas en inglés) es un indicador de momento que mide la magnitud de los movimientos recientes de precios para evaluar condiciones de sobrecompra o sobreventa en el precio de un activo. Se calcula utilizando la fórmula:

RSI=100− 100/(1+RS)
 

donde 
�
�
RS es la relación media de días de cierre hacia arriba entre la media de días de cierre hacia abajo. Para calcularlo para los últimos 20 y 10 días, necesitamos seguir los siguientes pasos para cada ventana de tiempo:

Calcular los cambios diarios en el precio de cierre.
Separar estos cambios en ganancias (aumentos) y pérdidas (disminuciones).
Calcular el promedio de ganancias y el promedio de pérdidas sobre el período de interés (20 días y 10 días, respectivamente).
Calcular el RS, que es el promedio de ganancias dividido por el promedio de pérdidas.
Utilizar la fórmula del RSI para obtener el valor final.

In [14]:
# Separar las ganancias (positivas) de las pérdidas (negativas)
gain = sp500['Momentum'].mask(sp500['Momentum'] > 0, 0)
loss = -sp500['Momentum'].mask(sp500['Momentum'] < 0, 0)

# Calcular el RSI para los últimos 20 y 10 días
window_lengths = [20, 10]

for length in window_lengths:
    # Calcular el promedio de ganancias y pérdidas
    avg_gain = gain.rolling(window=length, min_periods=1).mean()
    avg_loss = loss.rolling(window=length, min_periods=1).mean()
    
    # Calcular RS
    rs = avg_gain / avg_loss
    
    # Calcular RSI
    rsi = 100 - (100 / (1 + rs))
    
    # Adjuntar al DataFrame
    sp500[f'RSI_{length}'] = rsi

# Mostrar las últimas 20 filas para verificar las nuevas columnas
print(sp500.tail(20))



                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-01  5257.970215  5263.950195  5229.200195  5243.770020  3325930000   
2024-04-02  5204.290039  5208.339844  5184.049805  5205.810059  3886590000   
2024-04-03  5194.370117  5228.750000  5194.370117  5211.490234  3703250000   
2024-04-04  5244.049805  5256.589844  5146.060059  5147.209961  4075680000   
2024-04-05  5158.950195  5222.180176  5157.209961  5204.339844  3386780000   
2024-04-08  5211.370117  5219.569824  5197.350098  5202.390137  3278180000   
2024-04-09  5217.029785  5224.810059  5160.779785  5209.910156  3400680000   
2024-04-10  5167.879883  5178.430176  5138.700195  5160.640137  3845930000   
2024-04-11  5172.950195  5211.779785  5138.770020  5199.060059  3509380000   
2024-04-12  5171.509766  5175.029785  5107.939941  5123.410156  3963220000   
2024-04-15  5149.669922  5168.430176  5052.470215  5061.819824  


El indicador Larry Williams %R es una medida de momentum que se utiliza para identificar niveles de sobrecompra y sobreventa en un mercado. Se calcula como un porcentaje que refleja la posición del precio de cierre relativo a su rango más alto y más bajo durante un número específico de períodos pasados. La fórmula del Larry Williams %R es:

%R= [(Hn-C)/(Hn-Ln)]x100

donde:

C es el precio de cierre actual,
Hn  es el precio más alto en los últimos n períodos,
Ln es el precio más bajo en los últimos 
�
n períodos,
�
n es el número de períodos utilizados para el cálculo.
Generalmente, se utiliza un período de 14 días para el cálculo de este indicador, pero se puede ajustar según las necesidades del análisis. Para agregar el Larry Williams %R al DataFrame sp500 para los últimos 14 días, puedes usar el siguiente código:

In [15]:
# Asumiendo que sp500 es tu DataFrame y tiene columnas 'High', 'Low', y 'Close' con los precios más altos, más bajos y de cierre

# Configurar el número de períodos para el cálculo de Williams %R
n = 10

# Calcular el precio más alto y el más bajo de los últimos n días
high_n = sp500['High'].rolling(window=n).max()
low_n = sp500['Low'].rolling(window=n).min()

# Calcular el Williams %R
sp500['Williams_%R'] = ((high_n - sp500['Close']) / (high_n - low_n)) * -100

# Mostrar las últimas filas para verificar la nueva columna
print(sp500.tail())


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-22  4987.330078  5038.839844  4969.399902  5010.600098  3820250000   
2024-04-23  5028.850098  5076.120117  5027.959961  5070.549805  3751400000   
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   

            5d_moving_avg  10d_moving_avg  20d_moving_avg  200d_moving_avg  \
Date                                                                         
2024-04-22    5012.514063     5081.741064     5147.851562      4677.076304   
2024-04-23    5016.341992     5067.805029     5140.469556      4680.434302   
2024-04-24    5026.225977     5058.904004     5133.872046      4683.744802   
2024-04-25    5033.685938     5043.839990     5123.868530      

In [16]:
#https://school.stockcharts.com/doku.php?id=technical_indicators:moving_average_convergence_divergence_macd


# Calcular las EMAs de 12 y 26 períodos para la columna 'Close'
ema12 = sp500['Close'].ewm(span=12, adjust=False).mean()
ema26 = sp500['Close'].ewm(span=26, adjust=False).mean()

# Calcular el MACD restando la EMA de 26 períodos de la EMA de 12 períodos
sp500['MACD'] = ema12 - ema26

# Calcular la línea de señal como la EMA de 9 períodos del MACD
sp500['Signal_Line'] = sp500['MACD'].ewm(span=9, adjust=False).mean()

# Mostrar las últimas filas para verificar las nuevas columnas
print(sp500.tail())
sp500.head()


                   Open         High          Low        Close      Volume  \
Date                                                                         
2024-04-22  4987.330078  5038.839844  4969.399902  5010.600098  3820250000   
2024-04-23  5028.850098  5076.120117  5027.959961  5070.549805  3751400000   
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  3604140000   

            5d_moving_avg  10d_moving_avg  20d_moving_avg  200d_moving_avg  \
Date                                                                         
2024-04-22    5012.514063     5081.741064     5147.851562      4677.076304   
2024-04-23    5016.341992     5067.805029     5140.469556      4680.434302   
2024-04-24    5026.225977     5058.904004     5133.872046      4683.744802   
2024-04-25    5033.685938     5043.839990     5123.868530      

,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg,10d_wma,Momentum,Stochastic_%K,Stochastic_%D,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line
Date,,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,NaN,-55.799927,NaN,NaN,100.000000,100.000000,NaN,-4.451276,-0.890255
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,NaN,2.689941,NaN,NaN,95.401013,95.401013,NaN,-7.673434,-2.246891
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,NaN,1.339966,NaN,NaN,93.264385,93.264385,NaN,-10.003579,-3.798229
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,NaN,38.020020,NaN,NaN,57.026071,57.026071,NaN,-8.682254,-4.775034


In [17]:
# Calcular los retornos diarios
sp500['Daily_Returns'] = sp500['Close'].pct_change()

# Crear un "lag" de 1 día para los retornos diarios
sp500['Lag 1_Returns'] = sp500['Daily_Returns'].shift(+1)
sp500['Lag 2_Returns'] = sp500['Daily_Returns'].shift(+2)
sp500['Next_Day_Returns'] = sp500['Daily_Returns'].shift(-1)

# Crear la columna 'TARGET' basada en 'Lagged_Daily_Returns' - Quiero predecir si el retorno del día siguiente será positivo o negativo basado con respecto al día actual en el que tomo la decision al final del día
sp500['TARGET'] = np.where(sp500['Next_Day_Returns'] > 0, 1, 0)

# Mostrar las primeras filas para verificar las nuevas columnas
sp500.head()



,Open,High,Low,Close,Volume,5d_moving_avg,10d_moving_avg,20d_moving_avg,200d_moving_avg,10d_wma,...,RSI_20,RSI_10,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Next_Day_Returns,TARGET
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,1469.250000,1478.000000,1438.359985,1455.219971,931800000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,-0.038345,0
2000-01-04,1455.219971,1455.219971,1397.430054,1399.420044,1009000000,NaN,NaN,NaN,NaN,NaN,...,100.000000,100.000000,NaN,-4.451276,-0.890255,-0.038345,NaN,NaN,0.001922,1
2000-01-05,1399.420044,1413.270020,1377.680054,1402.109985,1085500000,NaN,NaN,NaN,NaN,NaN,...,95.401013,95.401013,NaN,-7.673434,-2.246891,0.001922,-0.038345,NaN,0.000956,1
2000-01-06,1402.109985,1411.900024,1392.099976,1403.449951,1092300000,NaN,NaN,NaN,NaN,NaN,...,93.264385,93.264385,NaN,-10.003579,-3.798229,0.000956,0.001922,-0.038345,0.027090,1
2000-01-07,1403.449951,1441.469971,1400.729980,1441.469971,1225200000,1420.333984,NaN,NaN,NaN,NaN,...,57.026071,57.026071,NaN,-8.682254,-4.775034,0.027090,0.000956,0.001922,0.011190,1


In [18]:
# Definimos una función que clasifica los retornos
def classify_returns(value):
    if value < -0.0043:
        return 0
    elif value <= 0.0062:
        return 1
    else:
        return 2

# Aplicamos la función para crear la nueva columna TARGET2
sp500['TARGET2'] = sp500['Next_Day_Returns'].apply(classify_returns)

print(sp500)

                   Open         High          Low        Close      Volume  \
Date                                                                         
2000-01-03  1469.250000  1478.000000  1438.359985  1455.219971   931800000   
2000-01-04  1455.219971  1455.219971  1397.430054  1399.420044  1009000000   
2000-01-05  1399.420044  1413.270020  1377.680054  1402.109985  1085500000   
2000-01-06  1402.109985  1411.900024  1392.099976  1403.449951  1092300000   
2000-01-07  1403.449951  1441.469971  1400.729980  1441.469971  1225200000   
...                 ...          ...          ...          ...         ...   
2024-04-22  4987.330078  5038.839844  4969.399902  5010.600098  3820250000   
2024-04-23  5028.850098  5076.120117  5027.959961  5070.549805  3751400000   
2024-04-24  5084.859863  5089.479980  5047.020020  5071.629883  3656740000   
2024-04-25  5019.879883  5057.750000  4990.580078  5048.419922  3958050000   
2024-04-26  5084.649902  5114.620117  5073.140137  5099.959961  

In [19]:
# Contar los valores nulos en cada columna
null_counts = sp500.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts)

# Eliminar las filas con valores nulos o NaN
sp500_clean = sp500.dropna()
# Contar los valores nulos en cada columna
null_counts_clean = sp500_clean.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

sp500_clean.to_csv("sp500_clean.csv")

Conteo de valores nulos por columna antes de limpiar:
 Open                  0
High                  0
Low                   0
Close                 0
Volume                0
5d_moving_avg         4
10d_moving_avg        9
20d_moving_avg       19
200d_moving_avg     199
10d_wma               9
Momentum              1
Stochastic_%K        19
Stochastic_%D        21
RSI_20                1
RSI_10                1
Williams_%R           9
MACD                  0
Signal_Line           0
Daily_Returns         1
Lag 1_Returns         2
Lag 2_Returns         3
Next_Day_Returns      1
TARGET                0
TARGET2               0
dtype: int64
Conteo de valores nulos por columna antes de limpiar:
 Open                0
High                0
Low                 0
Close               0
Volume              0
5d_moving_avg       0
10d_moving_avg      0
20d_moving_avg      0
200d_moving_avg     0
10d_wma             0
Momentum            0
Stochastic_%K       0
Stochastic_%D       0
RSI_20         

In [20]:
import requests


# FRED API endpoint
url = "https://api.stlouisfed.org/fred/series/observations"

# List of series IDs to be downloaded
# CREDIT RISK AND RISK APPETITE VARIABLES
# DAAA = Moody's Seasoned Aaa Corporate Bond Yield
# AAA10Y = Moody's Seasoned Aaa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# DBAA = Moody's Seasoned Baa Corporate Bond Yield
# BAA10Y = Moody's Seasoned Baa Corporate Bond Yield Relative to Yield on 10-Year Treasury Constant Maturity
# BAMLH0A0HYM2 = ICE BofA US High Yield Index Option-Adjusted Spread 
# BAMLH0A2HYB = ICE BofA Single-B US High Yield Index Option-Adjusted Spread 
# BAMLH0A3HYC = ICE BofA CCC & Lower US High Yield Index Option-Adjusted Spread
# BAMLHE00EHYIOAS = ICE BofA Euro High Yield Index Option-Adjusted Spread -  

# INTEREST RATE VARIABLES
# T10Y2Y = 10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity
# T10Y3M = 10-Year Treasury Constant Maturity Minus 3-Month Treasury Constant Maturity  
# T5YIE = 5-Year Breakeven Inflation Rate   

# EXCHANGE RATES
# DEXUSEU = U.S. Dollars to Euro Spot Exchange Rate - value of one U.S. Dollar in Euros
# DEXCHUS = Chinese Yuan Renminbi to U.S. Dollar Spot Exchange Rate - value of one Chinese Yuan Renminbi in U.S. Dollars
# DEXCAUS = Canadian Dollars to U.S. Dollar Spot Exchange Rate - value of one Canadian Dollar in U.S. Dollars
# DEXMXUS= Mexican Pesos to U.S. Dollar Spot Exchange Rate - value of one Mexican Peso in U.S. Dollars

# VOLATILITY INDECES
# VIXCLS = CBOE Volatility Index - VIX measures market expectation of near term volatility conveyed by stock index option prices.
#(GVZCLS)= CBOE Gold ETF Volatility Index 
series_ids = ['DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS','GVZCLS']

# Your FRED API key
api_key = '7de04d83472c6affd4e0a1505211d3b9'

# Initialize an empty dictionary to store DataFrames
dfs = {}

# Loop through each series ID
for series_id in series_ids:
    # API parameters
    params = {
        'series_id': series_id,
        'api_key': api_key,
        'file_type': 'json',
    }
    
    # Make the API request
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Convert the response to JSON
        data = response.json()
        # Extract data for DataFrame
        dates = [item['date'] for item in data['observations']]
        values = [item['value'] for item in data['observations']]
        # Create DataFrame and convert types
        temp_df = pd.DataFrame({'Date': dates, series_id: values})
        temp_df['Date'] = pd.to_datetime(temp_df['Date'])
        temp_df[series_id] = pd.to_numeric(temp_df[series_id], errors='coerce')
        # Add the DataFrame to the dictionary
        dfs[series_id] = temp_df.set_index('Date')
    else:
        print(f"Failed to retrieve data for {series_id}: {response.status_code}")

# Combine all the series DataFrames into one DataFrame
macroeconomic_df = pd.concat(dfs.values(), axis=1)




In [21]:
#Sacar los cambios diarios de las variables macroeconomicas

changes_daily = macroeconomic_df.diff()

# Renombrar las columnas para indicar que representan cambios diarios
changes_daily.columns = [f"{col}_daily_change" for col in macroeconomic_df.columns]

# Concatenar los cambios diarios de vuelta al DataFrame original
macroeconomic_df = pd.concat([macroeconomic_df, changes_daily], axis=1)


In [22]:
# Contar los valores nulos en cada columna
null_counts_2 =macroeconomic_df.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts)

# Eliminar las filas con valores nulos o NaN
macroeconomic_df_clean = macroeconomic_df.dropna()
# Contar los valores nulos en cada columna
null_counts_clean = macroeconomic_df_clean.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

macroeconomic_df_clean.to_csv("macroeconomic_df_clean.csv")

Conteo de valores nulos por columna antes de limpiar:
 Open                  0
High                  0
Low                   0
Close                 0
Volume                0
5d_moving_avg         4
10d_moving_avg        9
20d_moving_avg       19
200d_moving_avg     199
10d_wma               9
Momentum              1
Stochastic_%K        19
Stochastic_%D        21
RSI_20                1
RSI_10                1
Williams_%R           9
MACD                  0
Signal_Line           0
Daily_Returns         1
Lag 1_Returns         2
Lag 2_Returns         3
Next_Day_Returns      1
TARGET                0
TARGET2               0
dtype: int64
Conteo de valores nulos por columna antes de limpiar:
 DAAA                            0
AAA10Y                          0
DBAA                            0
BAA10Y                          0
BAMLH0A0HYM2                    0
BAMLH0A2HYB                     0
BAMLH0A3HYC                     0
BAMLHE00EHYIOAS                 0
T10Y2Y                       

In [23]:
# Unir macroeconomic_df_clean con sp500_clean basado en sus índices de fecha
AllVar_df = macroeconomic_df_clean.join(sp500_clean, how='inner')


# Contar los valores nulos en cada columna
null_counts_clean_AllVar = AllVar_df.isnull().sum()
print("Conteo de valores nulos por columna antes de limpiar:\n", null_counts_clean)

print(AllVar_df.index)
AllVar_df = AllVar_df.sort_index()

Conteo de valores nulos por columna antes de limpiar:
 DAAA                            0
AAA10Y                          0
DBAA                            0
BAA10Y                          0
BAMLH0A0HYM2                    0
BAMLH0A2HYB                     0
BAMLH0A3HYC                     0
BAMLHE00EHYIOAS                 0
T10Y2Y                          0
T10Y3M                          0
T5YIE                           0
DEXUSEU                         0
DEXCHUS                         0
DEXCAUS                         0
DEXMXUS                         0
VIXCLS                          0
GVZCLS                          0
DAAA_daily_change               0
AAA10Y_daily_change             0
DBAA_daily_change               0
BAA10Y_daily_change             0
BAMLH0A0HYM2_daily_change       0
BAMLH0A2HYB_daily_change        0
BAMLH0A3HYC_daily_change        0
BAMLHE00EHYIOAS_daily_change    0
T10Y2Y_daily_change             0
T10Y3M_daily_change             0
T5YIE_daily_change         

In [24]:
# Unir el DataFrame 'oil' al 'AllVar_df' que ya contiene datos combinados de 'macroeconomic_df' y 'sp500'
# Asegúrate de que 'oil' está indexado adecuadamente, si es necesario
if 'Date' in oil.columns:
    oil.set_index('Date', inplace=True)

# Especificando sufijos para resolver el conflicto de nombres de columnas
AllVar_df = AllVar_df.join(oil, how='inner', lsuffix='_AllVar', rsuffix='_oil')

# Mostrar las primeras filas para verificar las nuevas columnas
print(AllVar_df.head())

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  BAMLH0A2HYB  \
Date                                                                
2008-06-04  5.64    1.66  7.02    3.04          6.79         6.86   
2008-06-05  5.69    1.63  7.07    3.01          6.63         6.69   
2008-06-06  5.61    1.67  6.98    3.04          6.68         6.73   
2008-06-09  5.58    1.56  6.96    2.94          6.58         6.64   
2008-06-10  5.66    1.55  7.05    2.94          6.49         6.55   

            BAMLH0A3HYC  BAMLHE00EHYIOAS  T10Y2Y  T10Y3M  ...      MACD  \
Date                                                      ...             
2008-06-04        11.38             6.35    1.51    2.14  ... -0.616188   
2008-06-05        11.12             6.20    1.54    2.21  ...  0.498638   
2008-06-06        11.18             6.19    1.54    2.09  ... -2.093320   
2008-06-09        11.05             6.13    1.29    2.13  ... -4.014051   
2008-06-10        10.96             6.30    1.20    2.09  ... -5.7

In [25]:
# Unir el DataFrame 'oil' al 'AllVar_df' que ya contiene datos combinados de 'macroeconomic_df' y 'sp500'
# Asegúrate de que 'oil' está indexado adecuadamente, si es necesario
if 'Date' in oil.columns:
    oil.set_index('Date', inplace=True)

# Especificando sufijos para resolver el conflicto de nombres de columnas
AllVar_df = AllVar_df.join(oil, how='inner', lsuffix='_AllVar', rsuffix='_oil')

# Mostrar las primeras filas para verificar las nuevas columnas
print(AllVar_df.head())

            DAAA  AAA10Y  DBAA  BAA10Y  BAMLH0A0HYM2  BAMLH0A2HYB  \
Date                                                                
2008-06-04  5.64    1.66  7.02    3.04          6.79         6.86   
2008-06-05  5.69    1.63  7.07    3.01          6.63         6.69   
2008-06-06  5.61    1.67  6.98    3.04          6.68         6.73   
2008-06-09  5.58    1.56  6.96    2.94          6.58         6.64   
2008-06-10  5.66    1.55  7.05    2.94          6.49         6.55   

            BAMLH0A3HYC  BAMLHE00EHYIOAS  T10Y2Y  T10Y3M  ...  Daily_Returns  \
Date                                                      ...                  
2008-06-04        11.38             6.35    1.51    2.14  ...      -0.000327   
2008-06-05        11.12             6.20    1.54    2.21  ...       0.019496   
2008-06-06        11.18             6.19    1.54    2.09  ...      -0.030889   
2008-06-09        11.05             6.13    1.29    2.13  ...       0.000794   
2008-06-10        10.96             

In [26]:
# Se convierte el índice a tipo datetime
AllVar_df.index = pd.to_datetime(AllVar_df.index)
# Asegurar que el índice esté en formato de fecha y ordenado
AllVar_df.sort_index(inplace=True)

Variables control

Para los tres modelos probé iniciar el 30-Abril de  2014, 2019, 2022.  EL logit mejora  conforme utiliza menos periodo de tiempo. Random forest da mejor a partir del 2014 y emperoa si utilizamos 2019 y 2022 como puntos de inicio.  Para  guardar en csv los resutlados de cada estraegia de cada modelo con la fecha respectiva.

Selecciono el tamaño del periodo de prueba y la fecha de inicio de análisis para el que el periodo de prueba se mantenga como una serie de tiempo en orden histórico.

In [27]:
fecha_inicio = '2014-04-30'
test_size_Control=0.2

In [28]:
# Convertir a datetime
fecha_inicio_1 = pd.to_datetime(fecha_inicio)
end_date_1 = pd.to_datetime(end_date)


# Calcular el total de días
total_dias = (end_date_1 - fecha_inicio_1).days

# Calcular el número de días para el entrenamiento
dias_entrenamiento = int(total_dias * (1 - test_size_Control))

# Calcular la fecha de corte
fecha_corte = fecha_inicio_1 + pd.Timedelta(days=dias_entrenamiento)

print("Fecha de corte para dividir los datos:", fecha_corte)


Fecha de corte para dividir los datos: 2022-04-27 00:00:00


LOGIT

In [29]:
#  Pasar los nombres de las columnas a excluir como una lista al argumento `columns`


AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]

X_logit=AllVar_df_Shortened.drop(columns=['Next_Day_Returns','TARGET','TARGET2', 'DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS_daily_change','GVZCLS_daily_change'])

Y_logit= AllVar_df_Shortened['TARGET']

X_logit.head()

,VIXCLS,GVZCLS,DAAA_daily_change,AAA10Y_daily_change,DBAA_daily_change,BAA10Y_daily_change,BAMLH0A0HYM2_daily_change,BAMLH0A2HYB_daily_change,BAMLH0A3HYC_daily_change,BAMLHE00EHYIOAS_daily_change,...,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Close_oil_AllVar,oil_daily_changes_AllVar,Close_oil_oil,oil_daily_changes_oil
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,13.41,15.52,-0.03,0.01,-0.03,0.01,0.05,0.00,0.10,0.10,...,-3.189594,5.657878,3.440574,0.002992,0.004761,0.003236,99.739998,-1.520538,99.739998,-1.520538
2014-05-01,13.25,15.15,-0.05,-0.01,-0.06,-0.02,0.04,0.03,0.09,-0.01,...,-12.927632,6.344873,4.021434,-0.000143,0.002992,0.004761,99.419998,-0.320834,99.419998,-0.320834
2014-05-02,12.91,14.85,-0.04,-0.01,-0.03,0.00,-0.02,-0.01,-0.02,0.00,...,-25.024432,6.608187,4.538784,-0.001348,-0.000143,0.002992,99.760002,0.341988,99.760002,0.341988
2014-05-05,13.29,15.51,0.04,0.01,0.04,0.01,0.01,0.01,0.01,-0.02,...,-16.379977,7.019979,5.035023,0.001871,-0.001348,-0.000143,99.480003,-0.280672,99.480003,-0.280672
2014-05-06,13.80,15.02,-0.03,-0.01,-0.02,0.00,-0.02,-0.03,0.00,0.00,...,-57.981342,5.911264,5.210272,-0.008988,0.001871,-0.001348,99.500000,0.020101,99.500000,0.020101


In [30]:
#importo las libreris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [36]:

# Asumiendo que ya has definido X_logit (variables independientes) y Y_logit (variable dependiente), y están sin la columna 'Fecha'
X_train, X_test, Y_train, Y_test = train_test_split(X_logit, Y_logit, test_size=
test_size_Control, shuffle=False)  # shuffle=False para mantener el orden


scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Crear una instancia del modelo de Regresión Logística
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo con el conjunto de entrenamiento
model.fit(X_train_scaled, Y_train)



LogisticRegression(max_iter=1000)

In [37]:
# Realizar predicciones en el conjunto de prueba
Y_pred = model.predict(X_test_scaled )

In [38]:
# Imprimir reporte de clasificación
print(classification_report(Y_test, Y_pred))

# Imprimir matriz de confusión
print(confusion_matrix(Y_test, Y_pred))


              precision    recall  f1-score   support

           0       0.44      0.17      0.24       227
           1       0.51      0.80      0.62       242

    accuracy                           0.49       469
   macro avg       0.47      0.48      0.43       469
weighted avg       0.48      0.49      0.44       469

[[ 38 189]
 [ 48 194]]


In [39]:
Y_test.head()


Date
2022-04-28    0
2022-04-29    1
2022-05-03    1
2022-05-04    0
2022-05-05    0
Name: TARGET, dtype: int32

In [40]:
Y_pred_df=pd.DataFrame(Y_pred, columns=['Prediction'], index=Y_test.index)
Y_pred_df.head()

,Prediction
Date,
2022-04-28,1
2022-04-29,1
2022-05-03,1
2022-05-04,1
2022-05-05,1


In [41]:
result_LOGIT = pd.merge(Y_pred_df, Y_test, how='left', left_index=True, right_index=True)
result_LOGIT.head()

,Prediction,TARGET
Date,,
2022-04-28,1,0
2022-04-29,1,1
2022-05-03,1,1
2022-05-04,1,0
2022-05-05,1,0


In [42]:

# Formatear el nombre del archivo para incluir las variables 'fecha_inicio' y 'test_size_Control'
nombre_archivo = f'result_LOGIT_{fecha_inicio}_testsize{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV con el nombre formateado
result_LOGIT.to_csv(nombre_archivo, index=True)


RANDOM FOREST

In [43]:
from sklearn.ensemble import RandomForestClassifier  # O RandomForestRegressor si es un problema de regresión
from sklearn.metrics import accuracy_score, classification_report  # Importa otras métricas según tu caso


In [44]:
# Importar las librerías necesarias
from sklearn.ensemble import RandomForestClassifier

#AllVar_df =AllVar_df.loc['2019-04-03':].copy ()

#echa_inicio = '2012-01-01'
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]

# Definir las variables independientes X y la variable dependiente Y
X_RF = AllVar_df_Shortened.drop(columns=['Next_Day_Returns','TARGET','TARGET2'])
Y_RF = AllVar_df_Shortened['TARGET']


In [45]:
X_RF.head()

,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,BAMLH0A2HYB,BAMLH0A3HYC,BAMLHE00EHYIOAS,T10Y2Y,T10Y3M,...,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Close_oil_AllVar,oil_daily_changes_AllVar,Close_oil_oil,oil_daily_changes_oil
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,4.21,1.54,4.83,2.16,3.71,3.54,7.09,3.10,2.25,2.64,...,-3.189594,5.657878,3.440574,0.002992,0.004761,0.003236,99.739998,-1.520538,99.739998,-1.520538
2014-05-01,4.16,1.53,4.77,2.14,3.75,3.57,7.18,3.09,2.22,2.60,...,-12.927632,6.344873,4.021434,-0.000143,0.002992,0.004761,99.419998,-0.320834,99.419998,-0.320834
2014-05-02,4.12,1.52,4.74,2.14,3.73,3.56,7.16,3.09,2.18,2.58,...,-25.024432,6.608187,4.538784,-0.001348,-0.000143,0.002992,99.760002,0.341988,99.760002,0.341988
2014-05-05,4.16,1.53,4.78,2.15,3.74,3.57,7.17,3.07,2.20,2.60,...,-16.379977,7.019979,5.035023,0.001871,-0.001348,-0.000143,99.480003,-0.280672,99.480003,-0.280672
2014-05-06,4.13,1.52,4.76,2.15,3.72,3.54,7.17,3.07,2.18,2.58,...,-57.981342,5.911264,5.210272,-0.008988,0.001871,-0.001348,99.500000,0.020101,99.500000,0.020101


In [46]:
Y_RF.head()

Date
2014-04-30    0
2014-05-01    0
2014-05-02    1
2014-05-05    0
2014-05-06    1
Name: TARGET, dtype: int32

In [47]:
# Asegurar que el índice está en formato datetime (si es necesario)
X_RF.index = pd.to_datetime(X_RF.index)

# Asumiendo que Y_RF está alineado con X_RF
Y_RF.index = X_RF.index  # Asegúrate de que Y_RF tenga el mismo índice que X_RF

# Dividir los datos usando la fecha de corte
train_mask = X_RF.index <= fecha_corte
test_mask = X_RF.index > fecha_corte

X_train_RF = X_RF[train_mask]
Y_train_RF = Y_RF[train_mask]
X_test_RF = X_RF[test_mask]
Y_test_RF = Y_RF[test_mask]


In [48]:
# Dividir los datos en conjuntos de entrenamiento y prueba
#X_train_RF, X_test_RF, Y_train_RF, Y_test_RF = train_test_split(X_RF, Y_RF, test_size=test_size_Control, random_state=42)

scaler = StandardScaler()
scaler.fit(X_train_RF)
X_train_RF_scaled = scaler.transform(X_train_RF)
X_test_RF_scaled = scaler.transform(X_test_RF)

In [49]:
X_train_RF.head()


,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,BAMLH0A2HYB,BAMLH0A3HYC,BAMLHE00EHYIOAS,T10Y2Y,T10Y3M,...,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Close_oil_AllVar,oil_daily_changes_AllVar,Close_oil_oil,oil_daily_changes_oil
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,4.21,1.54,4.83,2.16,3.71,3.54,7.09,3.10,2.25,2.64,...,-3.189594,5.657878,3.440574,0.002992,0.004761,0.003236,99.739998,-1.520538,99.739998,-1.520538
2014-05-01,4.16,1.53,4.77,2.14,3.75,3.57,7.18,3.09,2.22,2.60,...,-12.927632,6.344873,4.021434,-0.000143,0.002992,0.004761,99.419998,-0.320834,99.419998,-0.320834
2014-05-02,4.12,1.52,4.74,2.14,3.73,3.56,7.16,3.09,2.18,2.58,...,-25.024432,6.608187,4.538784,-0.001348,-0.000143,0.002992,99.760002,0.341988,99.760002,0.341988
2014-05-05,4.16,1.53,4.78,2.15,3.74,3.57,7.17,3.07,2.20,2.60,...,-16.379977,7.019979,5.035023,0.001871,-0.001348,-0.000143,99.480003,-0.280672,99.480003,-0.280672
2014-05-06,4.13,1.52,4.76,2.15,3.72,3.54,7.17,3.07,2.18,2.58,...,-57.981342,5.911264,5.210272,-0.008988,0.001871,-0.001348,99.500000,0.020101,99.500000,0.020101


In [50]:
X_test_RF.head()


,DAAA,AAA10Y,DBAA,BAA10Y,BAMLH0A0HYM2,BAMLH0A2HYB,BAMLH0A3HYC,BAMLHE00EHYIOAS,T10Y2Y,T10Y3M,...,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns,Close_oil_AllVar,oil_daily_changes_AllVar,Close_oil_oil,oil_daily_changes_oil
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-28,3.99,1.14,4.91,2.06,3.83,4.09,8.13,4.50,0.22,2.03,...,-64.404044,-53.545916,-26.027430,0.024747,0.002098,-0.028146,105.360001,3.273872,105.360001,3.273872
2022-04-29,4.01,1.12,4.94,2.05,3.93,4.19,8.30,4.54,0.19,2.04,...,-98.031599,-65.322708,-33.886486,-0.036285,0.024747,0.002098,104.690002,-0.635913,104.690002,-0.635913
2022-05-03,4.11,1.14,5.02,2.05,3.97,4.20,8.71,4.64,0.19,2.06,...,-74.919524,-74.691468,-48.135163,0.004837,0.005675,-0.036285,102.410004,-2.624317,102.410004,-2.624317
2022-05-04,4.08,1.15,5.01,2.08,4.10,4.37,8.88,4.69,0.27,2.04,...,-47.237096,-66.053533,-51.718837,0.029862,0.004837,0.005675,107.809998,5.272916,107.809998,5.272916
2022-05-05,4.28,1.23,5.15,2.10,4.03,4.31,8.81,4.69,0.34,2.20,...,-73.908175,-70.762217,-55.527513,-0.035650,0.029862,0.004837,108.260002,0.417405,108.260002,0.417405


In [51]:
Y_train_RF.head()


Date
2014-04-30    0
2014-05-01    0
2014-05-02    1
2014-05-05    0
2014-05-06    1
Name: TARGET, dtype: int32

In [52]:
Y_test_RF.head()

Date
2022-04-28    0
2022-04-29    1
2022-05-03    1
2022-05-04    0
2022-05-05    0
Name: TARGET, dtype: int32

In [53]:
# Crear una instancia del modelo Random Forest
# Ajusta los hiperparámetros como n_estimators (número de árboles) y max_depth (profundidad máxima de cada árbol) según necesites
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo con el conjunto de entrenamiento escalado
rf_model.fit(X_train_RF_scaled, Y_train_RF)


RandomForestClassifier(random_state=42)

In [56]:
# Predecir las etiquetas para el conjunto de prueba
Y_pred_RF = rf_model.predict(X_test_RF_scaled)

# Calcular la precisión del modelo
accuracy = accuracy_score(Y_test_RF, Y_pred_RF)
print(f"Accuracy of the Random Forest model: {accuracy:.2f}")

# Otras métricas de evaluación
print(classification_report(Y_test_RF, Y_pred_RF))


Accuracy of the Random Forest model: 0.46
              precision    recall  f1-score   support

           0       0.47      0.81      0.60       227
           1       0.44      0.14      0.21       242

    accuracy                           0.46       469
   macro avg       0.45      0.48      0.40       469
weighted avg       0.45      0.46      0.40       469



In [57]:
Y_test_RF.head()

Date
2022-04-28    0
2022-04-29    1
2022-05-03    1
2022-05-04    0
2022-05-05    0
Name: TARGET, dtype: int32

In [58]:
#Transformo las predicciones en un DataFrame y agrupo resultados contra los valores reales
Y_pred_RF_df=pd.DataFrame(Y_pred_RF, columns=['Prediction'], index=Y_test_RF.index)
Y_pred_RF_df.head()


,Prediction
Date,
2022-04-28,0
2022-04-29,0
2022-05-03,0
2022-05-04,0
2022-05-05,1


In [59]:
result_RF = pd.merge(Y_pred_RF_df, Y_test_RF, how='left', left_index=True, right_index=True)
result_RF.head()

,Prediction,TARGET
Date,,
2022-04-28,0,0
2022-04-29,0,1
2022-05-03,0,1
2022-05-04,0,0
2022-05-05,1,0


In [60]:

# Formatear el nombre del archivo para incluir las variables 'fecha_inicio' y 'test_size_Control'
nombre_archivo_RF = f'result_RF_{fecha_inicio}_testsize{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV con el nombre formateado
result_RF.to_csv(nombre_archivo_RF, index=True)


In [62]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import pandas as pd
import numpy as np

# Suponiendo que X_RF y Y_RF ya están definidos y son tus datos originales
#X_train_RF_SV, X_test_RF_SV, Y_train_RF_SV, Y_test_RF_SV = train_test_split(X_RF, Y_RF, test_size=0.1, random_state=42)

X_train_RF_SV = X_RF[train_mask]
Y_train_RF_SV = Y_RF[train_mask]
X_test_RF_SV = X_RF[test_mask]
Y_test_RF_SV = Y_RF[test_mask]

# Escalar los datos
scaler = StandardScaler()
scaler.fit(X_train_RF_SV)  # Asegurarte de ajustar el scaler a X_train_RF_SV
X_train_RF_SV_scaled = scaler.transform(X_train_RF_SV)
X_test_RF_SV_scaled = scaler.transform(X_test_RF_SV)

# Verificar si X_train_RF_SV es un DataFrame y extraer los nombres de las columnas
if isinstance(X_train_RF_SV, pd.DataFrame):
    feature_names = X_train_RF_SV.columns.tolist()
else:
    raise ValueError("X_train_RF_SV debe ser un DataFrame para extraer nombres de columnas.")

# Convertir arrays de NumPy a DataFrames para facilitar la manipulación
X_train_RF_SV_scaled_df = pd.DataFrame(X_train_RF_SV_scaled, columns=feature_names)
X_test_RF_SV_scaled_df = pd.DataFrame(X_test_RF_SV_scaled, columns=feature_names)

# Crear el modelo de Random Forest
model = RandomForestClassifier(n_estimators=50, random_state=42)

# Crear el objeto RFE y clasificar las características más importantes
rfe = RFE(estimator=model, n_features_to_select=10)
rfe.fit(X_train_RF_SV_scaled_df, Y_train_RF_SV)

# Crear un DataFrame para mostrar los resultados usando los nombres de las características
results_df = pd.DataFrame({
    'Feature': feature_names,
    'Selected': rfe.support_,
    'Ranking': rfe.ranking_
})

# Filtrar para mostrar sólo las características seleccionadas
selected_features_df = results_df[results_df['Selected'] == True]

# Imprimir las características seleccionadas y su ranking
print("Selected features and their rankings:")
print(selected_features_df)


Selected features and their rankings:
                  Feature  Selected  Ranking
11                DEXUSEU      True        1
16                 GVZCLS      True        1
28   DEXUSEU_daily_change      True        1
32    VIXCLS_daily_change      True        1
38                 Volume      True        1
43                10d_wma      True        1
52          Daily_Returns      True        1
53          Lag 1_Returns      True        1
54          Lag 2_Returns      True        1
58  oil_daily_changes_oil      True        1


In [64]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def optimize_rf_with_feature_selection(X_train_opt, y_train_opt):
    if isinstance(X_train_opt, np.ndarray):
        X_train_opt = pd.DataFrame(X_train_opt, columns=['feature_' + str(i) for i in range(X_train_opt.shape[1])])

    # Definir los parámetros para la búsqueda
    params = {
        'n_estimators': (10, 1000),
        'max_depth': (1, 100),
        'min_samples_split': (2, 20),
        'min_samples_leaf': (1, 20),
        'max_features': ('sqrt', 'log2', 0.1, 0.5, 0.9),  # Ajuste de parámetros válidos
    }
    
    clf = RandomForestClassifier(random_state=42)
    selector = SelectFromModel(clf)
    opt = BayesSearchCV(clf, params, n_iter=50, cv=3, random_state=42)
    opt.fit(X_train_opt, y_train_opt)

    best_estimator = opt.best_estimator_
    selector.estimator = best_estimator
    selector.fit(X_train_opt, y_train_opt)
    X_selected = selector.transform(X_train_opt)
    selected_features = X_train_opt.columns[selector.get_support()]

    return best_estimator, selected_features, X_selected

# Preparación y escalado de datos
#X_train_RF_opt, X_test_RF_opt, y_train_RF_opt, y_test_RF_opt = train_test_split(X_RF, Y_RF, test_size=0.2, random_state=42)
X_train_RF_opt = X_RF[train_mask]
Y_train_RF_opt = Y_RF[train_mask]
X_test_RF_opt = X_RF[test_mask]
Y_test_RF_opt = Y_RF[test_mask]

scaler = StandardScaler()
X_train_RF_opt_scaled = scaler.fit_transform(X_train_RF_opt)
X_test_RF_opt_scaled = scaler.transform(X_test_RF_opt)

# Optimización y selección de características
best_estimator, selected_features, X_train_RF_opt_scaled_selected = optimize_rf_with_feature_selection(X_train_RF_opt_scaled, Y_train_RF_opt)

# Entrenamiento del modelo final
X_train_RF_opt_scaled_selected_df = pd.DataFrame(X_train_RF_opt_scaled_selected, columns=selected_features)
best_estimator.fit(X_train_RF_opt_scaled_selected_df, Y_train_RF_opt)

# Preparación del conjunto de prueba
X_test_RF_opt_scaled_selected = pd.DataFrame(X_test_RF_opt_scaled, columns=[f'feature_{i}' for i in range(X_test_RF_opt_scaled.shape[1])])[selected_features]

# Evaluación del modelo
y_pred_RF_opt = best_estimator.predict(X_test_RF_opt_scaled_selected)
accuracy = accuracy_score(Y_test_RF_opt, y_pred_RF_opt)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(Y_test_RF_opt, y_pred_RF_opt))
print("Confusion Matrix:")
print(confusion_matrix(Y_test_RF_opt, y_pred_RF_opt))


Accuracy: 0.52
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.03      0.05       227
           1       0.52      0.98      0.68       242

    accuracy                           0.52       469
   macro avg       0.53      0.50      0.36       469
weighted avg       0.53      0.52      0.37       469

Confusion Matrix:
[[  6 221]
 [  5 237]]


In [67]:
Y_test_RF_opt.head()

Date
2022-04-28    0
2022-04-29    1
2022-05-03    1
2022-05-04    0
2022-05-05    0
Name: TARGET, dtype: int32

In [68]:
y_pred_RF_opt_df=pd.DataFrame(y_pred_RF_opt, columns=['Prediction'], index=Y_test_RF_opt.index)   
y_pred_RF_opt_df.head()

,Prediction
Date,
2022-04-28,1
2022-04-29,1
2022-05-03,1
2022-05-04,1
2022-05-05,1


In [70]:
result_RF_opt = pd.merge(y_pred_RF_opt_df, Y_test_RF_opt, how='left', left_index=True, right_index=True)
result_RF_opt.head()

,Prediction,TARGET
Date,,
2022-04-28,1,0
2022-04-29,1,1
2022-05-03,1,1
2022-05-04,1,0
2022-05-05,1,0


In [71]:

# Formatear el nombre del archivo para incluir las variables 'fecha_inicio' y 'test_size_Control'
nombre_archivo_RF_opt = f'result_RF_opt_{fecha_inicio}_testsize{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV con el nombre formateado
result_RF_opt.to_csv(nombre_archivo_RF_opt, index=True)


LSTM

In [72]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from tensorflow.keras.utils import to_categorical

In [76]:
# Definir las variables independientes X y la variable dependiente Y
#X_LSTM = AllVar_df.drop(columns=['Next_Day_Returns','TARGET', 'DAAA','AAA10Y','DBAA','BAA10Y','BAMLH0A0HYM2','BAMLH0A2HYB','BAMLH0A3HYC','BAMLHE00EHYIOAS','T10Y2Y','T10Y3M','T5YIE','DEXUSEU','DEXCHUS','DEXCAUS','DEXMXUS','VIXCLS_daily_change','GVZCLS_daily_change'])
#fecha_inicio = '2012-01-01'
#['Open', 'High', 'Low', 'Close', 'Volume', 'DAAA', 'AAA10Y', 'DBAA', 'BAA10Y', 'BAMLH0A0HYM2', 'BAMLH0A2HYB', 'BAMLH0A3HYC', 'BAMLHE00EHYIOAS', 'T10Y2Y', 'T10Y3M', 'T5YIE', 'DEXUSEU', 'DEXCHUS', 'DEXCAUS', 'DEXMXUS', 'VIXCLS', 'GVZCLS', 'DAAA_daily_change', 'AAA10Y_daily_change', 'DBAA_daily_change', 'BAA10Y_daily_change', 'BAMLH0A0HYM2_daily_change', 'BAMLH0A2HYB_daily_change', 'BAMLH0A3HYC_daily_change', 'BAMLHE00EHYIOAS_daily_change', 'T10Y2Y_daily_change', 'T10Y3M_daily_change', 'T5YIE_daily_change', 'DEXUSEU_daily_change', 'DEXCHUS_daily_change', 'DEXCAUS_daily_change', 'DEXMXUS_daily_change', 'VIXCLS_daily_change', 'GVZCLS_daily_change', 'Close_oil', 'oil_daily_changes']
AllVar_df_Shortened = AllVar_df.loc[fecha_inicio:]
X_LSTM = AllVar_df_Shortened[['Volume', 'DAAA_daily_change', 'AAA10Y_daily_change', 'DBAA_daily_change' ,'BAA10Y_daily_change', 'BAMLH0A0HYM2_daily_change','BAMLH0A2HYB_daily_change','BAMLH0A3HYC_daily_change','BAMLHE00EHYIOAS_daily_change', 'T10Y2Y_daily_change', 'T10Y3M_daily_change', 'T5YIE_daily_change' ,'DEXUSEU_daily_change' ,'DEXCHUS_daily_change', 'DEXCAUS_daily_change','DEXMXUS_daily_change', 'VIXCLS_daily_change' ,'GVZCLS_daily_change' ,'oil_daily_changes_oil','20d_moving_avg','200d_moving_avg','10d_wma','Stochastic_%K','Stochastic_%D','RSI_20','Williams_%R','MACD','Signal_Line','Daily_Returns','Lag 1_Returns','Lag 2_Returns']]
Y_LSTM = AllVar_df_Shortened['TARGET']


In [77]:
X_LSTM.head()

,Volume,DAAA_daily_change,AAA10Y_daily_change,DBAA_daily_change,BAA10Y_daily_change,BAMLH0A0HYM2_daily_change,BAMLH0A2HYB_daily_change,BAMLH0A3HYC_daily_change,BAMLHE00EHYIOAS_daily_change,T10Y2Y_daily_change,...,10d_wma,Stochastic_%K,Stochastic_%D,RSI_20,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,3779230000,-0.03,0.01,-0.03,0.01,0.05,0.00,0.10,0.10,-0.02,...,1870.665654,83.924179,75.828069,50.324568,-3.189594,5.657878,3.440574,0.002992,0.004761,0.003236
2014-05-01,3416740000,-0.05,-0.01,-0.06,-0.02,0.04,0.03,0.09,-0.01,-0.03,...,1872.956190,83.598687,81.556479,51.524745,-12.927632,6.344873,4.021434,-0.000143,0.002992,0.004761
2014-05-02,3159560000,-0.04,-0.01,-0.03,0.00,-0.02,-0.01,-0.02,0.00,-0.04,...,1875.081106,80.535449,82.686105,51.608552,-25.024432,6.608187,4.538784,-0.001348,-0.000143,0.002992
2014-05-05,2733730000,0.04,0.01,0.04,0.01,0.01,0.01,0.01,-0.02,0.02,...,1876.158196,91.334384,85.156173,45.490976,-16.379977,7.019979,5.035023,0.001871,-0.001348,-0.000143
2014-05-06,3327260000,-0.03,-0.01,-0.02,0.00,-0.02,-0.03,0.00,0.00,-0.02,...,1875.627461,69.325719,80.398517,44.698477,-57.981342,5.911264,5.210272,-0.008988,0.001871,-0.001348


In [78]:
# Asegurar que el índice está en formato datetime (si es necesario)
X_LSTM.index = pd.to_datetime(X_LSTM.index)

# Asumiendo que Y_RF está alineado con X_RF
Y_LSTM.index = X_LSTM.index  # Asegúrate de que Y_RF tenga el mismo índice que X_RF

# Dividir los datos usando la fecha de corte
train_mask_LSTM = X_LSTM.index <= fecha_corte
test_mask_LSTM = X_LSTM.index > fecha_corte

X_train_LSTM = X_LSTM[train_mask_LSTM]
Y_train_LSTM = Y_LSTM[train_mask_LSTM]
X_test_LSTM = X_LSTM[test_mask_LSTM]
Y_test_LSTM = Y_LSTM[test_mask_LSTM]


In [79]:
X_train_LSTM.head()

,Volume,DAAA_daily_change,AAA10Y_daily_change,DBAA_daily_change,BAA10Y_daily_change,BAMLH0A0HYM2_daily_change,BAMLH0A2HYB_daily_change,BAMLH0A3HYC_daily_change,BAMLHE00EHYIOAS_daily_change,T10Y2Y_daily_change,...,10d_wma,Stochastic_%K,Stochastic_%D,RSI_20,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2014-04-30,3779230000,-0.03,0.01,-0.03,0.01,0.05,0.00,0.10,0.10,-0.02,...,1870.665654,83.924179,75.828069,50.324568,-3.189594,5.657878,3.440574,0.002992,0.004761,0.003236
2014-05-01,3416740000,-0.05,-0.01,-0.06,-0.02,0.04,0.03,0.09,-0.01,-0.03,...,1872.956190,83.598687,81.556479,51.524745,-12.927632,6.344873,4.021434,-0.000143,0.002992,0.004761
2014-05-02,3159560000,-0.04,-0.01,-0.03,0.00,-0.02,-0.01,-0.02,0.00,-0.04,...,1875.081106,80.535449,82.686105,51.608552,-25.024432,6.608187,4.538784,-0.001348,-0.000143,0.002992
2014-05-05,2733730000,0.04,0.01,0.04,0.01,0.01,0.01,0.01,-0.02,0.02,...,1876.158196,91.334384,85.156173,45.490976,-16.379977,7.019979,5.035023,0.001871,-0.001348,-0.000143
2014-05-06,3327260000,-0.03,-0.01,-0.02,0.00,-0.02,-0.03,0.00,0.00,-0.02,...,1875.627461,69.325719,80.398517,44.698477,-57.981342,5.911264,5.210272,-0.008988,0.001871,-0.001348


In [80]:
Y_train_LSTM.head()

Date
2014-04-30    0
2014-05-01    0
2014-05-02    1
2014-05-05    0
2014-05-06    1
Name: TARGET, dtype: int32

In [81]:
X_test_LSTM.head()

,Volume,DAAA_daily_change,AAA10Y_daily_change,DBAA_daily_change,BAA10Y_daily_change,BAMLH0A0HYM2_daily_change,BAMLH0A2HYB_daily_change,BAMLH0A3HYC_daily_change,BAMLHE00EHYIOAS_daily_change,T10Y2Y_daily_change,...,10d_wma,Stochastic_%K,Stochastic_%D,RSI_20,Williams_%R,MACD,Signal_Line,Daily_Returns,Lag 1_Returns,Lag 2_Returns
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-28,4854180000,0.02,-0.01,0.04,0.01,-0.06,-0.06,-0.06,-0.05,-0.02,...,4371.841815,28.307272,10.957410,66.232373,-64.404044,-53.545916,-26.027430,0.024747,0.002098,-0.028146
2022-04-29,5084030000,0.02,-0.02,0.03,-0.01,0.10,0.10,0.17,0.04,-0.03,...,4355.980034,1.630621,11.489401,68.909331,-98.031599,-65.322708,-33.886486,-0.036285,0.024747,0.002098
2022-05-03,4582050000,-0.07,-0.05,-0.05,-0.03,-0.08,-0.10,0.02,0.05,-0.07,...,4297.955522,21.277344,13.466518,69.481730,-74.919524,-74.691468,-48.135163,0.004837,0.005675,-0.036285
2022-05-04,5136360000,-0.03,0.01,-0.01,0.03,0.13,0.17,0.17,0.05,0.08,...,4257.531490,51.817254,30.195396,60.113308,-47.237096,-66.053533,-51.718837,0.029862,0.004837,0.005675
2022-05-05,5077030000,0.20,0.08,0.14,0.02,-0.07,-0.06,-0.07,0.00,0.07,...,4224.582191,18.393128,30.495909,63.683403,-73.908175,-70.762217,-55.527513,-0.035650,0.029862,0.004837


In [82]:
Y_test_LSTM.head()

Date
2022-04-28    0
2022-04-29    1
2022-05-03    1
2022-05-04    0
2022-05-05    0
Name: TARGET, dtype: int32

In [83]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Supongamos que tienes un DataFrame llamado X_LSTM con características y otro llamado Y_LSTM con etiquetas

# Dividir los datos en conjuntos de entrenamiento y prueba (80% entrenamiento, 20% prueba)
#X_train_LSTM, X_test_LSTM, Y_train_LSTM, Y_test_LSTM = train_test_split(X_LSTM, Y_LSTM, test_size=0.2, random_state=42)

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador a los datos de entrenamiento y transformarlos
X_train_scaled_LSTM = scaler.fit_transform(X_train_LSTM)
X_test_scaled_LSTM = scaler.transform(X_test_LSTM)

# Cambiar la forma de X_train y X_test para LSTM: [muestras, pasos de tiempo, características]
X_train_LSTM_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_LSTM_reshaped= X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Ahora tienes los conjuntos de entrenamiento y prueba listos para usar en tu red LSTM


In [ ]:
#NO UTIIZAR ESTE CODIGO. ES UN LTSM DE DOS CAPAS.  DA MEJOR RESULTADOS EL DE TRES CAPAS QUE ESTA ABAJO
 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    # Primera capa LSTM con return_sequences=True para permitir más capas LSTM
    LSTM(50, activation='relu', input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]), return_sequences=True),
    Dropout(0.2),  # Dropout para evitar el sobreajuste

    # Segunda capa LSTM
    LSTM(50, activation='relu', return_sequences=False),  # Cambio clave para permitir más capas LSTM
    Dropout(0.2),

    # Capa densa final para clasificación binaria
    Dense(1, activation='sigmoid')  # Función de activación sigmoid para salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    # Primera capa LSTM con return_sequences=True para permitir más capas LSTM
    LSTM(50, activation='relu', input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]), return_sequences=True),
    Dropout(0.2),  # Dropout para evitar el sobreajuste

    # Segunda capa LSTM
    LSTM(50, activation='relu', return_sequences=True),  # Cambio clave para permitir más capas LSTM
    Dropout(0.2),

    # Tercera capa LSTM (última capa)
    LSTM(50, activation='relu', return_sequences=False),  # Cambio clave para la última capa LSTM
    Dropout(0.2),

    # Capa densa final para clasificación binaria
    Dense(1, activation='sigmoid')  # Función de activación sigmoid para salida binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


c:\Users\lgarn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [85]:
# Entrenar el modelo
history = model.fit(X_train_LSTM_reshaped, Y_train_LSTM, epochs=20, batch_size=64, validation_data=(X_test_LSTM_reshaped, Y_test_LSTM))


Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5398 - loss: 0.6928 - val_accuracy: 0.5160 - val_loss: 0.6927
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5408 - loss: 0.6916 - val_accuracy: 0.5160 - val_loss: 0.6927
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5386 - loss: 0.6907 - val_accuracy: 0.5160 - val_loss: 0.6930
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5359 - loss: 0.6897 - val_accuracy: 0.5160 - val_loss: 0.6945
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5501 - loss: 0.6850 - val_accuracy: 0.5160 - val_loss: 0.6954
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5549 - loss: 0.6836 - val_accuracy: 0.5160 - val_loss: 0.6967
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5427 - loss: 0.6827 - val_accuracy: 0.5160 - val_loss: 0.6974
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5629 - loss: 0.6805 - val_accuracy: 0.5096 - val_loss

In [86]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(X_test_LSTM_reshaped, Y_test_LSTM)
print(f'Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}')


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 866us/step - accuracy: 0.4995 - loss: 0.8208
Test Accuracy: 0.510, Test Loss: 0.768


In [87]:
# Evaluar el modelo para obtener pérdida y precisión
test_loss, test_acc = model.evaluate(X_test_LSTM_reshaped, Y_test_LSTM)
print(f'Test Accuracy: {test_acc:.3f}, Test Loss: {test_loss:.3f}')

# Obtener las predicciones del modelo
predictions = model.predict(X_test_LSTM_reshaped)

# Si deseas, puedes imprimir algunas de las predicciones para visualizarlas
print(predictions[:10])  # Imprime las primeras 10 predicciones


15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 687us/step - accuracy: 0.4995 - loss: 0.8208
Test Accuracy: 0.510, Test Loss: 0.768
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step
[[0.49979204]
 [0.35431442]
 [0.35860428]
 [0.6740569 ]
 [0.6680588 ]
 [0.5344033 ]
 [0.85740435]
 [0.22625504]
 [0.03639165]
 [0.30016303]]


### NO PUEDO EXTRAER RESULTADOS LSTM

In [88]:
# Crear un DataFrame con las predicciones y los valores reales
resultados = pd.DataFrame({
    'Predicciones': predictions.flatten(),  # Asegúrate de aplanar las predicciones si son multidimensionales
    'Valores_Reales': Y_test_LSTM.flatten()  # Aplanar si es necesario
})

# Construir el nombre del archivo incluyendo las variables
nombre_archivo = f'resultados_{fecha_inicio}_testSize_{test_size_Control}.csv'

# Guardar el DataFrame en un archivo CSV
resultados.to_csv(nombre_archivo, index=False)


AttributeError: 'Series' object has no attribute 'flatten'

Definir la función para construir el modelo
Vamos a definir una función build_model que toma un argumento hp (de hiperparámetros) y construye el modelo LSTM con hiperparámetros variables.

In [ ]:
from tensorflow import keras
from kerastuner.tuners import RandomSearch

def build_model(hp):
    model = Sequential([
        LSTM(
            hp.Int('units_1', min_value=30, max_value=100, step=10),
            activation=hp.Choice('activation_1', ['relu', 'tanh']),
            input_shape=(X_train_LSTM_reshaped.shape[1], X_train_LSTM_reshaped.shape[2]),
            return_sequences=True
        ),
        Dropout(hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)),

        LSTM(
            hp.Int('units_2', min_value=30, max_value=100, step=10),
            activation=hp.Choice('activation_2', ['relu', 'tanh']),
            return_sequences=True
        ),
        Dropout(hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)),

        LSTM(
            hp.Int('units_3', min_value=30, max_value=100, step=10),
            activation=hp.Choice('activation_3', ['relu', 'tanh']),
            return_sequences=False
        ),
        Dropout(hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)),

        Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model
